**事前準備**

1. [nightley](https://nightley.jp/archives/1954/) から東京都全域の境界データをダウンロード（.zipファイル）
2. .zipファイルを解凍し, `一般公開ファイル`を取得
3. 一般公開ファイル/20140808_snsbasedPeopleFlowData_nightley.zip を解凍

※ Kaggleにもバスの移動データ等が存在するらしい

In [ ]:
import os
import folium
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

#### データ

【擬似人流データ　CSVデータ概要】

実データ：人々が訪れる場所や施設、時間帯(SNS ベース)、道路ネットワーク  
推定データ：移動経路、居住地、勤務地、滞在時間、性別、滞在中何をしているか(買い 物やレジャーなど)  
含まれないデータ：鉄道経路、個人を特定する情報、滞在施設名  
エリア：東京近郊(首都圏)  
対象時期：2013年7月(7/1, 7/7)、10月(10/7, 10/13)、12月(12/16, 12/22)  
時間単位：5分毎(24時間分)  
レコード数：各csvファイルで約70,000件  
ファイルサイズ：約100MB  

---

【ファイル構成】

疑似人流データ(csv形式)  

- 1行目：ユーザー ID
- 2行目：性別(推定値) ※文字列型
- 3行目：日付・時刻
- 4行目：緯度
- 5行目：経度
- 6行目：滞在者カテゴリ(大分類) ※文字列型
- 7行目：滞在者カテゴリ(小分類) ※文字列型
- 8行目：状態(滞在or移動) ※文字列型
- 9行目：滞在者カテゴリID(6行目に対応)  


In [ ]:
df_all = pd.read_csv("./一般公開ファイル/20140808_snsbasedPeopleFlowData_nightley/2013-07-01.csv",header=None)
df_all.columns=["uuid","sex","date","lat","lon","level1","level2","state","level2id"]

In [ ]:
df_all.head(2)

In [ ]:
from folium.plugins import HeatMap,HeatMapWithTime

# folium icon list
# https://fontawesome.com/search?q=train&o=r&s=solid&f=classic

#### 全データを可視化

In [ ]:
geo_list = df_all[["lat","lon"]].values.tolist()

In [ ]:
fmap = folium.Map(
    location=[35.6809591,139.7673068], 
    zoom_start=14, 
    tiles='cartodbdark_matter',
    prefer_canvas=True,
)

In [ ]:
HeatMap(geo_list,radius=2,blur=15).add_to(fmap)

In [ ]:
fmap

#### あるUUIDのデータを可視化

In [ ]:
df = df_all[df_all["uuid"]==1013]

In [ ]:
df.shape

In [ ]:
geo_list = df[["lat","lon"]].values.tolist()

In [ ]:
fmap = folium.Map(
    location=[35.6809591,139.7673068], 
    zoom_start=14, 
    tiles='cartodbdark_matter',
    prefer_canvas=True,
)

In [ ]:
folium.Marker(
    location=[35.6809591,139.7673068],
    icon=folium.Icon(
        color="darkred",icon="train",icon_color="white",prefix="fa",
        # popup="Tokyo Station", tooltip='Tokyo Station',
    )
).add_to(fmap)

In [ ]:
HeatMap(geo_list,radius=10,blur=5).add_to(fmap)

In [ ]:
fmap

In [ ]:
# fmap.save('demo_map.html')

#### あるUUIDのデータを時系列で可視化

In [ ]:
df = df_all[df_all["uuid"]==1013]

In [ ]:
geo_list = df[["lat","lon"]].values.tolist()

In [ ]:
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d %H:%M")

In [ ]:
fmap = folium.Map(
    location=[35.6809591,139.7673068], 
    zoom_start=16, 
    tiles='cartodbdark_matter',
    prefer_canvas=True,
)
folium.Marker(
    location=[35.6809591,139.7673068],
    icon=folium.Icon(
        color="darkred",icon="train",icon_color="white",prefix="fa",
        # popup="Tokyo Station", tooltip='Tokyo Station',
    )
).add_to(fmap)

In [ ]:
# HeatMapWithTime用にデータを加工（時間軸を考慮した配列構造）
geo_loc_list = [loc for loc in geo_list]

In [ ]:
from datetime import datetime, timedelta

HeatMapWithTime(geo_loc_list,radius=25,index=df["date"].tolist(),auto_play=True,max_opacity=0.6).add_to(fmap)

In [ ]:
fmap

In [ ]:
fmap.save("time_demo.html")

#### 参考

- [Pythonライブラリのfoliumでサクッとクールなヒートマップを作ってみる](https://note.com/narudesu/n/n3d882a711e24)
- [python foliumで時系列ヒートマップ(time series heatmap)描く](https://sammi-baba.hatenablog.com/entry/2018/12/25/074017)
- [Simple GPS data visualization using Python and Open Street Maps](https://towardsdatascience.com/simple-gps-data-visualization-using-python-and-open-street-maps-50f992e9b676): 画像に描画する方式